# AV235

Updating the dataset: P103030100000

* LiF1 is the guide channel.
* LiF1, SiC1, SiC2 count rates are rock solid.  
* LiF2 count rate dips, particularly in exposures 1-4.  
* Rescale LiF 2A to match LiF 1B, avoiding the worm.

COMMENT File updated 4 August 2022<br>
COMMENT LiF 2A scaled to match LiF 1B.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E12

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Read header keywords from FUSE *all* file.

filename = 'P103030100000all4ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# The guide channel is LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed spectra.  
# Note that spectra are scaled to have values near unity.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_ylim([0,2])

# Everything looks good except for LiF 2B.  We don't use this segment, so won't bother rescaling.

In [ ]:
# Let's see what's happening at long wavelengths.

f, ax = plt.subplots()
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()

# Again, LiF 2A is fainter, but LiF 1B suffers from the worm.

In [ ]:
# We'll scale LiF 2A to match LiF 1B at short wavelengths.

# Select a spectral region.

region = SpectralRegion(1100*u.AA, 1140*u.AA)
sub_lif2a = extract_region(lif2a, region)
sub_lif1b = extract_region(lif1b, region)

# Compute ratio of their fluxes.

scale_lif2a = sub_lif1b.mean()/sub_lif2a.mean()
print ('Scale LiF 2A by', scale_lif2a)

# Rescale LiF 2A.

lif2a *= scale_lif2a
lif2a_bsmooth *= scale_lif2a

In [ ]:
# How does this look?

f, ax = plt.subplots()
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()

In [ ]:
# Compare LiF 2A with a STIS spectrum of the same star.

filename = 'oe9f1s020_x1d.fits'
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.set_xlim([1120,1200])
ax.set_ylim([0,2])
ax.legend()

print ('Target:  ', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))

# Looks pretty good.

In [ ]:
# Just to check, is everyone happy with the SiC spectra?

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.legend()
ax.set_xlim([960, 1020])
ax.set_ylim([0,2])

# Looks good.

In [ ]:
# Apply these corrections to the NVO file.

filename = 'P103030100000nvo4ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale LiF 2A 
g = np.where((data['wave'] > 1090) & (data['wave'] < 1180))
data['flux'][g] *= float(scale_lif2a)
data['error'][g] *= float(scale_lif2a)

hdr['comment'] = ''
hdr['comment'] = 'File updated 4 August 2022'
hdr['comment'] = 'LiF 2A scaled to match LiF 1B.'

f.writeto('level0_P103030100000nvo4ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'P103030100000nvo4ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 

filename = 'level0_P103030100000nvo4ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(old['wave'], old['flux'], label='OLD FLUX')
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux']) 
ax2.step(new['wave'], new['flux'])
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,2E-12])